# Reconocimiento de patrones: Preparación de datos
### Ramón Soto C. [(rsotoc@moviquest.com)](mailto:rsotoc@moviquest.com/)
![ ](images/blank.png)
![agents](images/binary_data_under_a_magnifying.jpg)
[ver en nbviewer](http://nbviewer.ipython.org/github/rsotoc/pattern-recognition/blob/master/Limpieza%20de%20datos%20I.ipynb)

## Definición

Los datos son la base de la nueva economía de la información. Cada día se generan 2.5 x 10<sup>18</sup> bytes de datos ([aquí](http://www.vcloudnews.com/every-day-big-data-statistics-2-5-quintillion-bytes-of-data-created-daily/) un interesante *infographic* al respecto), provenientes de sensores, GPSs, redes sociales, mensajes electrónicos, transacciones comerciales, publicaciones regulares, etc. Esos datos permiten generar una gran cantidad de información para atender virtualmente cualquier problema. Sin embargo, antes de poder explotar la información contenida en ellos y antes de poder generar conocimiento de utilidad para la toma de decisiones, es necesario garantizar que los datos se encuentren en 'buenas condiciones'. 

Es una estimación bien conocida en tre los científicos de datos que el 80% del tiempo dedicado a la solución de un problema se invierte en la preparación de los datos: 

![](images/time.jpg)
![ ](images/blank.png)

El proceso de mejoramiento de los datos es lo que se denomina **preparación de los datos**. 
![](images/DataPreparation.png)
![ ](images/blank.png)
* La *limpieza de datos* consiste en rellenar valores faltantes, suavizar datos con ruido, identificar y remover valor atípicos y resolver inconsistencias. 
* La *integración de datos* es la integración de diversas fuentes de datos: bases de datos, cubos de datos o archivos. 
* La *selección de datos* consiste en seleccionar el conjunto de datos adecuado para analizar el sistema, incluyendo el muestreo. 
* La *selección de características* es un proceso mediante el cual se analizan las variables determinantes para describir los datos. 
* La *transformación de datos* incluye operaciones como normalización, agregación, codificación. 



## Limpieza de los datos 

### Valores faltantes 

El problema de valores faltantes es un problema muy frecuente al tratar de realizar cualquier tarea de análisis de datos y puede deberse a diversas razones: 
* Fallas en los mecanismos de medición (sensores defectuosos, por ejemplo) 
* Integración de conjuntos de datos no bien coordinados (mediciones con diferentes ciclos, por ejemplo) 
* Variables nuevas no consideradas o no disponibles originalmente 
* Respuestas omitidas intencionalmente por la fuente 

![](images/missingData.png)
![ ](images/blank.png)

La omisión de valores en el conjunto de datos puede tener diversos efectos y diferentes grados de impacto. En términos generales, se suelen considerar los siguientes grados de impacto, dependiendo del porcentaje de valores faltantes (*dumb rules*):
La omisión de valores en el conjunto de datos puede tener diversos efectos y diferentes grados de impacto. En términos generales, se suelen considerar los siguientes grados de impacto, dependiendo del porcentaje de valores faltantes (*dumb rules*):
* Menos de 1%: Trivial (no relevante)
* 1-5%: Manejable
* 5-15%: Manejable mediante métodos sofisticados
* Más de 15%: Crítico, con impacto severo en cualquier tipo de interpretación

Considérese el siguiente conjunto de datos tomados del conjunto de datos de diabetes:


In [14]:
"""
Reconocimiento de patrones: Limpieza de datos
"""

import numpy as np
import pandas as pd
import os

os.chdir('Data sets/Pima Indian Data Set') 

OSError: [Errno 2] No such file or directory: 'Data sets/Pima Indian Data Set'

In [13]:

df = pd.read_csv("pima-indians-diabetes.data", 
                 names = ['emb', 'gl2h', 'pad', 'ept', 'is2h', 'imc', 'fpd', 'edad', 'class'])

print(df.describe())
print 

print(df)

              emb        gl2h         pad         ept        is2h         imc  \
count  768.000000  768.000000  768.000000  768.000000  768.000000  768.000000   
mean     3.845052  120.894531   69.105469   20.536458   79.799479   31.992578   
std      3.369578   31.972618   19.355807   15.952218  115.244002    7.884160   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      1.000000   99.000000   62.000000    0.000000    0.000000   27.300000   
50%      3.000000  117.000000   72.000000   23.000000   30.500000   32.000000   
75%      6.000000  140.250000   80.000000   32.000000  127.250000   36.600000   
max     17.000000  199.000000  122.000000   99.000000  846.000000   67.100000   

              fpd        edad       class  
count  768.000000  768.000000  768.000000  
mean     0.471876   33.240885    0.348958  
std      0.331329   11.760232    0.476951  
min      0.078000   21.000000    0.000000  
25%      0.243750   24.000000    0.000000  
50%   

Como puede observarse, la variable *count* no es la misma para todas las columnas. Comparando con el despliegue de los datos, las diferencias en el valor de esta variable corresponde a los valores faltantes. Una mayor exploración podemos obtenerla de la siguiente manera:

In [ ]:
print ('Tabla de valores nulos')
print (df.isnull(), '\n')

print ('Contabilidad de valores nulos por columna')
print (df.isnull().sum() )

print ('Porcentaje de datos nulos en la columna *ept*')
eptNullPje = df['ept'].isnull().sum() / df.shape[0] * 100
print (eptNullPje)

Como puede apreciarse, el porcentaje de valores faltantes en este segmento de datos (45% de valores faltantes) está muy por encima de lo que puede tratarse de manera directa, según las reglas anteriores. 

En muchos casos, incluso detectar los valores faltantes es un problema. En nuestros datos originales, lo valores faltantes vienen enmascarados como 0, no como un espacio vacío. En este caso, el procedimiento anterior fallaría pues no existen datos 'no disponibles'. Debemos primero analizar los datos y detectar cómo se manifiestan los valores faltantes. En nuestro ejemplo, asumimos que *ept*, esto es, el 'Espesor de la piel del tríceps' no puede tener un valor de 0 y, por lo tanto, ese valor representa un valor faltante. Para resolver el problema, debemos preparar los datos asignando una etiqueta *NaN* a los valores que consideramos como valores'faltantes': 


In [ ]:
df2 = pd.read_csv("pima-indians-diabetes.data", 
                 names = ['emb', 'gl2h', 'pad', 'ept', 'is2h', 'imc', 'fpd', 'edad', 'class'])

print(df2)
print

df2.loc[df2['ept'] == 0,'ept'] = np.nan

print(df2)

En plataformas para *data science*, como R y Pandas, los valores marcados como 'NaN' suelen ser ignorados en las operaciones: 


In [ ]:
print( df2.info(), '\n')

print( df2.describe(), '\n')

print('Suma y promedio de ept: ({}, {})'.format(df2['ept'].sum(), df2['ept'].mean()), '\n')

print('Promedio tomando en cuenta los 0s:', df2['ept'].sum()/768 )


### Tratamiento de valores faltantes 

El método más simple para tratar con el problema de valores faltantes es la *eliminación de casos*, también conocido como *análisis de casos completos. Este método está disponible en todos los paquetes de análisis de datos y es la opción por omisión en la mayoría. 

En Pandas podemos eliminar los valores faltantes de diferentes maneras. *DataFrame.dropna*() elimina todos los renglones en el DataFrame en los que hay al menos un valor *NaN*:

In [ ]:
print(df)
print
print(df.dropna())

El parámetro *tresh* en DataFrame.dropna() permite eliminar todos los renglones que no contengan al menos el número de columnas "limpias" expresado por el prámetro. En los ejemplos a continuación, se conservan 1) sólo los renglones que tienen al menos 8 columnas *limpias* y 2) los renglones que tienen al menos 7 columnas con valores definidos:

In [ ]:
print(df.dropna(thresh=8), '\n')
print(df.dropna(thresh=7))

### Imputación 

El análisis de casos completos es una opción aceptable si el porcentaje de valores faltantes es pequeño. En la mayoría de los casos, es preferible reemplazar los valores faltantes por valores calculados por omisión o valores calculados. Esta operación se denomina **imputación**. En el siguiente ejemplo, todos los valores *NaN* son reemplazados por 0, lo cual pudiera seguir la lógica de que "si el dato no está disponible es que en realidad era cero.

In [ ]:
#print(df)
#print (df.describe())


print
print "imputacion por media"
dfx = df.fillna(df.mean() )
print(dfx)
print
print (dfx.describe())


print
print "datos sin imputacion"
df3 = pd.read_csv("pima-indians-diabetes.data", names = ['emb', 'gl2h', 'pad', 'ept', 'is2h', 'imc', 'fpd', 'edad', 'class'])    
print(df3)
print
print df3.describe()

Sin embargo, en muchos casos un valor por omisión de cero no tiene sentido. En nuestro ejemplo con los datos de diabetes, un valor de cero en la columna *pad* (*Presión diastólica de la sangre*) es imposible en una persona viva. En este caso, una mejor opción es rellenar los valores faltantes por el mínimo registrado:

In [ ]:
df3 = df.fillna(df.min())
print(df3, "\n")
print (df3.describe())

Otras alternativas comunes son rellenar los valores faltantes con el valor máximo o con valores estadísticos.

Reemplazar los valores faltantes por el valor promedio de esa variable es uno de los métodos más comunes de imputación, sin embargo la media es una medida vulnerable a valores atípicos. Una alternativa más robusta ante este problema es la mediana. 

In [ ]:
print "Rellenando con el valor máximo \n", df.fillna(df.max()).describe()
print "Rellenando con la media \n", df.fillna(df.mean()).describe()
print "Rellenando con la mediana \n", df.fillna(df.median()).describe()
print "Rellenando con la moda \n", df.fillna(df.mode()).describe()

Otra alternativa común es rellenar los valores faltantes con el valor no nulo previo o el siguiente:

In [ ]:
print (df)

print("Replicar hacia enfrente\n", df.fillna(method='pad'), "\n")
print("Replicar hacia atrás\n", df.fillna(method='bfill'))

Esta forma de tratar el problema de valores faltantes es muy común en análisis de series de tiempo. Esta estrategia suele designarse como *último valor conocido* y equivale a asumir que el sistema no pudo cambiar demasiado desde la última medición. En otros casos debe anaizarse si los datos realmente tienen una estructura local; esto es, determinar si tiene sentido esperar que los registros vecinos tengan valores cercanos. en el caso del conjunto de datos de diabetes, esta suposición no es válida.

Podemos también limitar el número de registros que son modificados de esta forma:

In [ ]:
print(df.fillna(method='pad', limit=1))

### Interpolación
La interpolación es un método formal para estimar valores en una serie de datos. La idea consiste en suponer que todos los puntos en la serie se encuentran sobre una curva subyacente, aunque desconocida. 
La forma más simple de interpolación es la *lineal*. En este caso se parte de dos puntos conocidos y los puntos intermedios (faltantes) se calculan como si estuvieran colocados sobre la línea recta que une a los puntos conocidos.

In [29]:
import matplotlib
import matplotlib.pyplot as plt


In [31]:
"""
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True, figsize=(10,10))
df['ept'].plot(ax=axes[0])
df['ept'].plot(ax=axes[0], grid=True, kind="bar")

dfi = df['ept'].interpolate()
dfi.plot(ax=axes[1])
dfi.plot(ax=axes[1], grid=True, kind="bar", color="green")
df['ept'].plot(ax=axes[1], grid=True, kind="bar")
"""

fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True, figsize=(50,50))
#df['ept'].plot(ax=axes[0])
#df['ept'].plot(ax=axes[0], grid=True, kind="bar")

"""
dfx['ept'].plot(ax=axes[0])
dfx['ept'].plot(ax=axes[0], grid=True, kind="bar")
plt.plot([1,2,3,4], [1,4,9,16], 'ro')
plt.axis([0, 6, 0, 20])
"""


#print (df['ept'].interpolate())

TypeError: 'AxesSubplot' object does not support indexing

La aproximación lineal, aunque es la más simple, es la menos natural. Es posible utilizar cualquier otro conjunto de curvas, típicamente de la forma, que se ajusten a los datos conocidos. A continuación se presentan ajustes a curvas cuadráticas y cúbicas:

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True, figsize=(10,10))
dfi = df['ept'].interpolate(method="quadratic")
dfi.plot(ax=axes[0])
dfi.plot(ax=axes[0], grid=True, kind="bar", color="red")
df['ept'].plot(ax=axes[0], grid=True, kind="bar")

dfi = df['ept'].interpolate(method="cubic")
dfi.plot(ax=axes[1])
dfi.plot(ax=axes[1], grid=True, kind="bar", color="red")
df['ept'].plot(ax=axes[1], grid=True, kind="bar")

Cualquier otra técnica de predicción puede ser empleada para rellenar los valores faltantes. Una de las más importantes es el razonamiento basado en casos ([RBC](https://en.wikipedia.org/wiki/Case-based_reasoning)). 

<hr style="border-width: 3px;">

### Tarea 2

* Analice los problemas de valores faltantes en el conjunto de datos *Pima Indians Diabetes* completo. 
* Realice la imputación de los datos utilizando 3 aproximaciones diferentes y compare los resultados.
* Realice una estimación de valores faltantes mediante interpolación.

**Fecha de entrega**: Martes 29 de agosto.
